# Introdução à Banco de Dados | Trabalho Prático III
Abel Severo Rocha, Ana Carla Fernandes, Natasha Caxias
Prof. Dr. Altigran Soares da Silva

## Parte I
Verificação dos parâmetros do hardware e do software utilizado e familiarização com o ambiente do PostgreSQL.

### Tarefa 1 - Identificação do Sistema

In [102]:
import platform
import os
import shutil
import subprocess

In [103]:
def mostrarSoInfo():
    
    soInfo = platform.freedesktop_os_release()
    print("\n--------- Sistema Operacional (SO) ---------\n")
    print("Sistema Operacional ", platform.system())
    print("Versão              ", soInfo["VERSION"])
    print("Release (kernel)    ", platform.release())
    print("Arquitetura         ", platform.machine())
    print("Distribuição        ", soInfo["NAME"])


In [104]:
def mostrarHardwareInfo():

    cpuInfo = {}
    with open("/proc/cpuinfo") as f:
        for line in f:
            if ":" in line:
                k, v = line.split(":", 1)
                cpuInfo[k.strip()] = v.strip()
    
    ramInfo = {}
    with open("/proc/meminfo") as f:
        for line in f:
            k, v = line.split(":", 1)
            ramInfo[k] = v.strip()

    print("\n------------- Sobre o Hardware -------------\n")
    print("Processador:")
    print("  • Modelo          ", cpuInfo["model name"])
    print("  • CPUs lógicas    ", os.cpu_count())
    print("  • Clock base      ", cpuInfo["cpu MHz"], "MHz")

    print("\nCache:")
    with open("/sys/devices/system/cpu/cpu0/cache/index0/size") as f:
        print("  • L1d             ", f.read().strip()[:-1]+" kB")
    with open("/sys/devices/system/cpu/cpu0/cache/index1/size") as f:
        print("  • L1i             ", f.read().strip()[:-1]+" kB")
    with open("/sys/devices/system/cpu/cpu0/cache/index2/size") as f:
        print("  • L2              ", f.read().strip()[:-1]+" kB")
    with open("/sys/devices/system/cpu/cpu0/cache/index3/size") as f:
        print("  • L3              ", f.read().strip()[:-1]+" kB")

    print("\nMemória RAM:")
    print("  • Total           ", ramInfo["MemTotal"])
    print("  • Livre           ", ramInfo["MemFree"])
    print("  • Disponível      ", ramInfo["MemAvailable"])

    total, usado, livre = shutil.disk_usage("/")
    print("\nDisco:")
    print(f"  • Total            {total/1024**3:.2f} GB")
    print(f"  • Usado            {usado/1024**3:.2f} GB")
    print(f"  • Livre            {livre/1024**3:.2f} GB")


In [105]:
mostrarSoInfo()
mostrarHardwareInfo()



--------- Sistema Operacional (SO) ---------

Sistema Operacional  Linux
Versão               24.04.3 LTS (Noble Numbat)
Release (kernel)     6.8.0-87-generic
Arquitetura          x86_64
Distribuição         Ubuntu

------------- Sobre o Hardware -------------

Processador:
  • Modelo           11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz
  • CPUs lógicas     8
  • Clock base       1663.902 MHz

Cache:
  • L1d              48 kB
  • L1i              32 kB
  • L2               1280 kB
  • L3               12288 kB

Memória RAM:
  • Total            7670432 kB
  • Livre            892220 kB
  • Disponível       2428748 kB

Disco:
  • Total            233.18 GB
  • Usado            41.99 GB
  • Livre            179.28 GB


### Tarefa 2 - Verificação de parâmetros de armazenamento

In [ ]:
BASE = "/sys/block"

def run(cmd):
    return subprocess.check_output(cmd, text=True).strip()

def getDispositivo(BASE):
    for dev in os.listdir(BASE):
        if dev.startswith(("loop", "ram", "dm-")): # ignora
            continue
        return dev # encontra dispositivo
    return None

def ehHD(dispositivo):
    rotational_path = os.path.join(BASE, dispositivo, "queue/rotational")
    
    if os.path.exists(rotational_path):
        with open(rotational_path) as f:
            return f.read().strip() == "1"
    return False


Tipo:  nvme
Setor Físico:  512 bytes
Setor Lógico:  512 bytes
